In [ ]:
import pandas as pd
import os

data_root = 'data/O3-Bench'
jsonl_file = f'{data_root}/test/metadata.jsonl'

df = pd.read_json(jsonl_file, orient='records', lines=True)
df['file_name'] = df['file_name'].apply(lambda x: os.path.join(f'{data_root}/test', x))
df.head()

,file_name,subset,question,options,answer,explanation,target_layouts,layout_infos,img_width,img_height
0,data/O3-Bench/test/images/map/festival_ADF2025...,map,A visitor at the Artist Market booth 233 needs...,A. area of grid B4\nB. area of grid B5\nC. are...,C,Step 1. Refer to the legend [layout 3] to iden...,"[1, 2, 3]","{'1': {'layout_id': 1, 'bbox': [2066.0, 17.996...",8000,5125
1,data/O3-Bench/test/images/map/mall_notice_stit...,map,"If I am at Zoff and want to go to BRIDGE, what...","A. Escalator C, 5F\nB. Elevator C, 5F\nC. Esca...",A,Step 1. Refer to the Legend [layout2] to ident...,"[2, 4, 5]","{'1': {'layout_id': 1, 'bbox': [10.0472663139,...",7121,5898
2,data/O3-Bench/test/images/map/subway_Interior-...,map,Which station provides long-term and free dail...,A. Doraville\nB. North Springs\nC. Arts Center...,A,"Step 1. In layout 2, locate the icons for long...","[1, 2]","{'1': {'layout_id': 1, 'bbox': [19.5, 500.5, 5...",6500,6500
3,data/O3-Bench/test/images/map/subway_chengdu-m...,map,Starting at 'Fengxihe' and ending at 'Financia...,A. Huaxing\nB. Luhu\nC. Tianfu Park\nD. Luomas...,D,"Step 1. Locate Line 4 and Line 17, which pass ...","[3, 4, 5]","{'1': {'layout_id': 1, 'bbox': [10.4895, 10.46...",6993,6015
4,data/O3-Bench/test/images/map/campus_morningsi...,map,"Starting from Lerner Hall, walk north past Tea...",A. Broadway → 121st Street\nB. Broadway → 120t...,A,Step 1. Locate Lerner Hall in the bottom-left ...,"[2, 4, 8]","{'1': {'layout_id': 1, 'bbox': [15.1245551601,...",4250,5500


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import textwrap
import numpy as np

# Global variables
filtered_df = None
current_index = 0
output_dir = None

# Output widgets
img_out = widgets.Output()
info_out = widgets.Output()

# Control widgets
index_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(df)-1 if len(df) > 0 else 0,
    step=1,
    description='Index:',
    style={'description_width': 'initial'}
)

show_layout_checkbox = widgets.Checkbox(
    value=True,
    description='Show Layout Boxes',
    style={'description_width': 'initial'}
)

# Navigation buttons
prev_button = widgets.Button(
    description='Previous',
    button_style='info',
    layout=widgets.Layout(width='100px')
)

next_button = widgets.Button(
    description='Next',
    button_style='info',
    layout=widgets.Layout(width='100px')
)

status_label = widgets.Label(value='Ready')

def format_long_text(text, width=80):
    """Format long text with line wrapping"""
    if not text:
        return 'N/A'
    
    # Use textwrap to wrap lines
    wrapped_lines = textwrap.wrap(str(text), width=width)
    return '\n'.join(wrapped_lines)

def go_previous():
    """Go to previous item"""
    if index_slider.value > index_slider.min:
        index_slider.value = index_slider.value - 1

def go_next():
    """Go to next item"""
    if index_slider.value < index_slider.max:
        index_slider.value = index_slider.value + 1

# Update function
def update_display(change=None):
    global current_index, filtered_df
    current_index = index_slider.value
    
    if filtered_df is None:
        filtered_df = df  # Use the full dataframe initially
    
    if filtered_df is None or current_index < 0 or current_index >= len(filtered_df):
        with img_out:
            clear_output(wait=True)
            print('No data available.')
        with info_out:
            clear_output(wait=True)
            print('No information available.')
        return
    
    row = filtered_df.iloc[current_index]
    
    # Display image with layout boxes
    with img_out:
        clear_output(wait=True)
        try:
            img_path = row['file_name']
            print(f"Trying to load image: {img_path}")
            
            if not os.path.exists(img_path):
                print(f"Image not found: {img_path}")
                # Check if file exists with different extension or path
                base_path = os.path.splitext(img_path)[0]
                for ext in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']:
                    alt_path = base_path + ext
                    if os.path.exists(alt_path):
                        print(f"Found alternative: {alt_path}")
                        img_path = alt_path
                        break
                else:
                    print("No alternative image found")
                    return
            # Load and display image
            img = Image.open(img_path)
            img_array = np.array(img)
            print(f"Image loaded successfully. Size: {img.size}")
            
            # Create figure with image display
            fig, ax_img = plt.subplots(1, 1, figsize=(20, 15))
            
            # Display image
            ax_img.imshow(img_array)
            ax_img.set_title(f"Index {current_index}: {os.path.basename(img_path)}", fontsize=16, weight='bold')
            ax_img.axis('off')
            
            # Draw bboxes for layouts if enabled
            if show_layout_checkbox.value and 'layout_infos' in row and row['layout_infos']:
                target_layouts = row.get('target_layouts', [])
                layout_infos = row['layout_infos']
                
                for layout_id, layout_info in layout_infos.items():
                    bbox = layout_info['bbox']  # [xmin, ymin, xmax, ymax]
                    
                    # Convert layout_id to int for comparison
                    layout_id_int = int(layout_id)
                    
                    # Use different colors for target vs non-target layouts
                    if layout_id_int in target_layouts:
                        color = 'red'
                        linewidth = 3
                    else:
                        color = 'blue'
                        linewidth = 2
                    
                    # Draw rectangle
                    rect = patches.Rectangle((bbox[0], bbox[1]), 
                                           bbox[2] - bbox[0], 
                                           bbox[3] - bbox[1],
                                           linewidth=linewidth, 
                                           edgecolor=color, 
                                           facecolor='none')
                    ax_img.add_patch(rect)
                    
                    # Add layout ID text (just the ID number)
                    ax_img.text(bbox[0] + 5, bbox[1] + 20, 
                               layout_id, 
                               fontsize=12, 
                               color=color, 
                               weight='bold',
                               bbox=dict(boxstyle="round,pad=0.3", 
                                       facecolor='white', 
                                       alpha=0.8))
            
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"Error loading image: {e}")
            import traceback
            traceback.print_exc()
    
    # Display information
    text_width = 120
    with info_out:
        clear_output(wait=True)
        print(f"Index: {current_index}")
        print("\n=== Question ===")
        
        # Combine question with options from the options column if it exists
        question = row.get('question', '')
        for q in question.split('\n'):
            print(format_long_text(q, width=int(text_width*1.5)))
        print(row.get('options'))
        
        print("\n=== Answer ===")
        print(row.get('answer', 'N/A'))
        if 'target_layouts' in row:
            print("\n=== Target Layouts ===")
            print(row['target_layouts'])
        if 'explanation' in row:
            print("\n=== Explanation ===")
            explanation = row.get('explanation', '')
            for ex in explanation.split('\n'):
                print(format_long_text(ex, width=int(text_width*1.5)))


In [ ]:
index_slider.observe(update_display, names='value')
show_layout_checkbox.observe(update_display, names='value')
prev_button.on_click(lambda x: go_previous())
next_button.on_click(lambda x: go_next())

# Create UI - put text area below image area and make both full width
ui = widgets.VBox([
    widgets.HBox([index_slider, show_layout_checkbox, prev_button, next_button]),
    img_out,  # Image area takes full width
    info_out,  # Text area takes full width and placed below image
    widgets.HBox([index_slider, show_layout_checkbox, prev_button, next_button]),
], layout=widgets.Layout(width='100%'))

# Initialize display
print("Initializing visualizer interface...")
update_display()
# Display the interface
display(ui)